## ***Installing conda-colab***

In [ ]:
# Install Conda on colab

# !conda --version

# !pip install -q condacolab
# import condacolab
# condacolab.install()

# !conda --version

# **Below code is based on torchreid.**

### ***Reffrence: [Deep Person Re-ID](https://github.com/KaiyangZhou/deep-person-reid)***

In [11]:
# !git clone https://github.com/KaiyangZhou/deep-person-reid.git
!git clone https://github.com/salidotir/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9862, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 9862 (delta 249), reused 288 (delta 194), pack-reused 9487
Receiving objects: 100% (9862/9862), 9.59 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (7257/7257), done.


In [ ]:
!pwd

/content


In [12]:
# cd to your preferred directory and clone this repo
# create environment
%cd deep-person-reid
!conda create --name torchreid python=3.7
!conda activate torchreid

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
# !conda install pytorch torchvision cudatoolkit=9.0 -c pytorch
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

/content/deep-person-reid
/bin/bash: conda: command not found
/bin/bash: conda: command not found
     |████████████████████████████████| 5.6 MB 8.0 MB/s 
     |████████████████████████████████| 73 kB 2.3 MB/s 
     |████████████████████████████████| 185 kB 79.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 68 kB 9.3 MB/s 
     |████████████████████████████████| 41 kB 781 kB/s 
/bin/bash: conda: command not found
/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
Compiling torchreid/metrics/rank_cylib/rank_cy.pyx because it changed.
[1/1] Cythonizing torchreid/metrics/rank_cylib/rank_cy.pyx
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a la

In [13]:
# Import torchreid
import torchreid

# # Load data manager
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3516 KB/s, 42 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************


~.~.~.~.~.~.~.~.~.~.~.~.~.~.


In [ ]:

# Build model, optimizer and lr_scheduler
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

model = model.cuda()

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3188 KB/s, 46 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************


~.~.~.~.~.~.~.~.~.~.~.~.~.~.


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



~.~.~.~.~.~.~.~.~.~.~.~.~.~.
~.~.~.~.~.~.~.~.~.~.~.~.~.~.
~.~.~.~.~.~.~.~.~.~.~.~.~.~.
~.~.~.~.~.~.~.~.~.~.~.~.~.~.
~.~.~.~.~.~.~.~.~.~.~.~.~.~.


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Run training & test
# engine.run(
#     save_dir='log/resnet50',
#     max_epoch=60,
#     eval_freq=10,
#     print_freq=10,
#     test_only=False
# )

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")

# Run test only

# Mount Google Drive
# from google.colab import drive
# drive.mount("/content/drive")

# Load a trained model
# weight_path = "/content/drive/My Drive/Colab Notebooks/Re-ranking Person Re-ID/resnet50_market_xent.pth.tar"

# torchreid.utils.load_pretrained_weights(model, weight_path)

engine.run(
    save_dir='log/resnet50',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True
)

print("~.~.~.~.~.~.~.~.~.~.~.~.~.~.")


~.~.~.~.~.~.~.~.~.~.~.~.~.~.
##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-2048 matrix
Speed: 0.0212 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 2.4%
CMC curve
Rank-1  : 8.1%
Rank-5  : 16.4%
Rank-10 : 22.5%
Rank-20 : 29.3%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "log/resnet50/visrank_market1501" ...
~.~.~.~.

## ***Use Torchreid as a feature extractor in your projects***

In [18]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path='/content/drive/My Drive/Colab Notebooks/Re-ranking Person Re-ID/osnet_x1_0_imagenet.pth',
    device='cuda'
)
!pwd
image_list = [
    '/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/0001_c1s1_001051_00.jpg',
    '/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/0001_c2s1_000301_00.jpg',
    '/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/0001_c3s1_000551_00.jpg',
    '/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/0001_c4s6_000810_00.jpg',
    '/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/0001_c5s1_001426_00.jpg'
]

features = extractor(image_list)
print(features.shape) # output (5, 512)
print(features[1].shape)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "/content/drive/My Drive/Colab Notebooks/Re-ranking Person Re-ID/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
/content/deep-person-reid
torch.Size([5, 512])
torch.Size([512])


## ***Working with 'result.csv' file***
In this part, we are going to read result.csv file and extract features of data.

In [22]:
import csv

# Open & read csv file
with open('/content/drive/My Drive/result.csv') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  line_count = 0
  for row in csv_reader:
    # each row -> [q_idx, g_idx, invalid]
    tmp_q_idx = int(row[0][1:])
    tmp_g_idx = int(row[1][1:])
    # print(int(row[0][1:]))
    # print(int(row[1][1:]))
    # print(bool(row[2][1:-2]))
    line_count += 1
  # print(f'Processed {line_count} lines.')


# Read from query/gallery dataset by q_idx, g_idx
# From previous cell result we know that:
num_of_query = 3368
num_of_gallery = 15913
my_dataset = datamanager.fetch_test_loaders('market1501')
my_query, my_gallery = my_dataset
assert num_of_query == len(my_query)
assert num_of_gallery == len(my_gallery)

qimg_path, qpid, qcamid = my_query[tmp_q_idx][:3]
print(qimg_path, qpid, qcamid)
gimg_path, gpid, gcamid = my_gallery[tmp_g_idx][:3]
print(gimg_path, gpid, gcamid)

# extract features of qimg
feature = extractor([qimg_path])
print(feature[0])

/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/query/1498_c3s3_062353_00.jpg 1498 2
/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15/bounding_box_test/0155_c5s1_077898_02.jpg 155 4
tensor([[0.0000e+00, 3.0809e+00, 0.0000e+00, 0.0000e+00, 2.8883e+00, 0.0000e+00,
         1.1778e+00, 1.9512e-01, 1.3726e+00, 8.7426e-01, 0.0000e+00, 7.9306e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0613e-02, 0.0000e+00, 0.0000e+00,
         1.5960e+00, 0.0000e+00, 1.4814e+00, 4.1426e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0484e+00, 1.5563e+00, 2.5087e+00, 0.0000e+00, 8.0976e-01,
         7.1853e-01, 0.0000e+00, 0.0000e+00, 6.8635e-01, 0.0000e+00, 1.0677e-01,
         9.5716e-01, 2.4810e+00, 1.5689e+00, 1.5088e+00, 1.9559e+00, 2.4324e+00,
         0.0000e+00, 0.0000e+00, 9.9673e-03, 0.0000e+00, 4.2695e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.9312e+00, 1.6580e+00, 6.4535e-01,
         2.6159e+00, 1.4790e+00, 0.0000e+00, 2.9